# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [2]:
url = 'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado'

r = requests.get(url)
print(f'Status Code: {r.status_code}')

data_json = r.json()
df_rj = pd.DataFrame(data_json['items'])

Status Code: 200


In [4]:
df_rj.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,13560,Estado,Rio de Janeiro,33,RJ,None,Deferido,19405.000026/2002-01,2002-06-25T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,49942.0,1,0,27/06/2002
1,5918,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000233/2012-05,2012-04-16T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Corporação Andina de Fomento,Dólar dos EUA,319675000.0,1,0,11/03/2016
2,13624,Estado,Rio de Janeiro,33,RJ,None,Deferido,19405.000039/2002-72,2002-06-20T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,67482400.0,1,0,27/06/2002
3,10562,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001228/2013-92,2013-10-22T02:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,500000000.0,1,0,27/12/2013
4,12375,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001671/2011-00,2011-10-18T02:00:00Z,Operação contratual externa (com garantia da U...,Preservação e recuperação ambiental,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,451980000.0,1,0,27/12/2011


In [5]:
# tres status mais frequentes
status_top_3 = df_rj['status'].value_counts()
print('Três status mais frequentes:\n')
print(status_top_3.nlargest(3))

Três status mais frequentes:

status
Encaminhado à PGFN com manifestação técnica favorável    34
Deferido                                                 14
Arquivado a pedido                                        3
Name: count, dtype: int64


In [6]:
# 3) Seu código aqui
df_rj['data'] = pd.to_datetime(df_rj['data_status'], format='%d/%m/%Y')
df_rj['ano'] = df_rj['data'].dt.year
df_rj['ano'].value_counts()

ano
2012    11
2009     9
2013     7
2011     6
2002     4
2016     4
2014     4
2010     3
2006     2
2008     2
Name: count, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [7]:
#1) Seu código aqui

def get_pvl(estado: str, tipo_interessado: str) -> pd.DataFrame:

    try:
        url = f'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={estado}&tipo_interessado={tipo_interessado}'

        r = requests.get(url)
        print(f'Status Code: {r.status_code}')

        data_json = r.json()
        print(data_json)

        df = pd.DataFrame(data_json['items'])
        
        return df
    except Exception as error:
        print(error)

In [8]:
df_MG = get_pvl('MG', 'Estado')
df_MG.head()

Status Code: 200
{'items': [{'id_pleito': 13602, 'tipo_interessado': 'Estado', 'interessado': 'Minas Gerais', 'cod_ibge': 31, 'uf': 'MG', 'num_pvl': None, 'status': 'Encaminhado à PGFN com manifestação técnica favorável', 'num_processo': '19405.000034/2005-92', 'data_protocolo': '2005-10-31T02:00:00Z', 'tipo_operacao': 'Operação contratual externa (com garantia da União)', 'finalidade': 'Infraestrutura', 'tipo_credor': 'Instituição Financeira Internacional', 'credor': 'Banco Interamericano de Desenvolvimento', 'moeda': 'Dólar dos EUA', 'valor': 50000000, 'pvl_assoc_divida': 1, 'pvl_contradado_credor': 0, 'data_status': '18/04/2006'}, {'id_pleito': 13275, 'tipo_interessado': 'Estado', 'interessado': 'Minas Gerais', 'cod_ibge': 31, 'uf': 'MG', 'num_pvl': None, 'status': 'Arquivado a pedido', 'num_processo': '17944.002000/2013-10', 'data_protocolo': '2014-04-17T03:00:00Z', 'tipo_operacao': 'Operação contratual interna (com garantia da União)', 'finalidade': 'Infraestrutura', 'tipo_credor'

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,13602,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,19405.000034/2005-92,2005-10-31T02:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,50000000.0,1,0,18/04/2006
1,13275,Estado,Minas Gerais,31,MG,None,Arquivado a pedido,17944.002000/2013-10,2014-04-17T03:00:00Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,600000000.0,0,0,24/04/2014
2,11132,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001368/2009-84,2009-11-20T02:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,137000000.0,1,0,26/11/2009
3,11571,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001479/2009-91,2010-03-26T03:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,461000000.0,1,0,01/04/2010
4,11696,Estado,Minas Gerais,31,MG,None,Arquivado a pedido,17944.001504/2014-01,2015-04-13T03:00:00Z,Operação contratual externa (com garantia da U...,Segurança pública,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,70000000.0,0,0,29/04/2016


In [11]:
# 3) Seu código aqui
df_BA_muni = get_pvl('BA', 'Município')
df_BA_muni.head()

Status Code: 200
{'items': [{'id_pleito': 10467, 'tipo_interessado': 'Município', 'interessado': 'Abaíra', 'cod_ibge': 2900108, 'uf': 'BA', 'num_pvl': None, 'status': 'Arquivado', 'num_processo': '17944.001207/2008-18', 'data_protocolo': '2008-04-14T03:00:00Z', 'tipo_operacao': 'Operação contratual interna', 'finalidade': 'Caminho da Escola 2', 'tipo_credor': 'Instituição Financeira Nacional', 'credor': 'Banco do Brasil S/A', 'moeda': 'Real', 'valor': 299450, 'pvl_assoc_divida': 0, 'pvl_contradado_credor': 0, 'data_status': '26/08/2008'}, {'id_pleito': 10258, 'tipo_interessado': 'Município', 'interessado': 'Ubaitaba', 'cod_ibge': 2932200, 'uf': 'BA', 'num_pvl': None, 'status': 'Arquivado', 'num_processo': '17944.001156/2008-16', 'data_protocolo': '2008-04-10T03:00:00Z', 'tipo_operacao': 'Operação contratual interna', 'finalidade': 'Caminho da Escola 2', 'tipo_credor': 'Instituição Financeira Nacional', 'credor': 'Banco do Brasil S/A', 'moeda': 'Real', 'valor': 172700, 'pvl_assoc_divida

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,10467,Município,Abaíra,2900108,BA,None,Arquivado,17944.001207/2008-18,2008-04-14T03:00:00Z,Operação contratual interna,Caminho da Escola 2,Instituição Financeira Nacional,Banco do Brasil S/A,Real,299450.0,0,0,26/08/2008
1,10258,Município,Ubaitaba,2932200,BA,None,Arquivado,17944.001156/2008-16,2008-04-10T03:00:00Z,Operação contratual interna,Caminho da Escola 2,Instituição Financeira Nacional,Banco do Brasil S/A,Real,172700.0,0,0,13/08/2008
2,11720,Município,Mansidão,2920452,BA,None,Arquivado por decurso de prazo,17944.001510/2011-16,2011-11-29T02:00:00Z,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,1500000.0,0,0,21/01/2013
3,12143,Município,Ribeira do Amparo,2926509,BA,None,Arquivado por decurso de prazo,17944.001612/2011-23,2011-09-09T03:00:00Z,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,500000.0,0,0,15/01/2013
4,10904,Município,Macarani,2919702,BA,None,Deferido,17944.001311/2009-85,2010-02-22T03:00:00Z,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,65000.0,0,0,09/03/2010


In [10]:
# 4) Seu código aqui
df_BA = get_pvl('BA', 'Estado')

pvl_BA = df_BA.to_csv(index=False)

Status Code: 200
{'items': [{'id_pleito': 18550, 'tipo_interessado': 'Estado', 'interessado': 'Bahia', 'cod_ibge': 29, 'uf': 'BA', 'num_pvl': None, 'status': 'Deferido', 'num_processo': '19407.000048/2006-77', 'data_protocolo': '2006-07-11T03:00:00Z', 'tipo_operacao': 'Operação contratual interna', 'finalidade': 'Segurança pública', 'tipo_credor': 'Instituição Financeira Nacional', 'credor': 'Banco do Brasil S/A', 'moeda': 'Dólar dos EUA', 'valor': 457205.74, 'pvl_assoc_divida': 1, 'pvl_contradado_credor': 0, 'data_status': '20/07/2006'}, {'id_pleito': 10528, 'tipo_interessado': 'Estado', 'interessado': 'Bahia', 'cod_ibge': 29, 'uf': 'BA', 'num_pvl': None, 'status': 'Encaminhado à PGFN com manifestação técnica favorável', 'num_processo': '17944.001222/2010-72', 'data_protocolo': '2010-12-20T02:00:00Z', 'tipo_operacao': 'Operação contratual interna (com garantia da União)', 'finalidade': 'PAC - Copa Estádios', 'tipo_credor': 'Instituição Financeira Nacional', 'credor': 'Banco Nacional d